In [ ]:

import json
import jax.numpy as np
import plotly.graph_objects as go
import sys 
sys.path.append(' HTS')
import material_jcrit
pi = np.pi

with open('initfiles/init_args.json', 'r') as f:    # 传入地址
    args = json.load(f)
globals().update(args)


def Jcrit_strain(T, B, strain):
    if T > 60:
        jc,b,t = material_jcrit.get_critical_current(T, B, strain, "REBCO_HT")
    elif T <= 60:
        jc,b,t = material_jcrit.get_critical_current(T, B, strain, "REBCO_LT")
    return jc


def Jcrit_theta(Bx, By):
    k2 = 0.2**2
    beta = 0.65
    jc0 = 3e10
    B0 = 0.02
    jc = jc0 / (1 + (k2*Bx**2 + By**2)**0.5 / B0) ** beta
    return jc


77K, 各项异性公式与应变公式对比

In [ ]:
js, jtx, jty = np.zeros((10)), np.zeros((10)), np.zeros((10))
T = 77
strain = 0
for i in range(10):
    B = i+1e-5
    js0 = Jcrit_strain(T, B, 0)
    Bx, By = i, 0
    jtx0 = Jcrit_theta(Bx, By)
    Bx, By = 0, i
    jty0 = Jcrit_theta(Bx, By)
    js = js.at[i].set(js0)
    jtx = jtx.at[i].set(jtx0)
    jty = jty.at[i].set(jty0)

print(js)
print(jtx)
print(jty)

fig = go.Figure()
fig.add_scatter(x = np.arange(0, 10, 1), y = js, 
                    name = 'jcrit_strain', line = dict(width=5))
fig.add_scatter(x = np.arange(0, 10, 1), y = jtx, 
                    name = 'jcrit_theta_X', line = dict(width=5))
fig.add_scatter(x = np.arange(0, 10, 1), y = jty, 
                    name = 'jcrit_theta_y', line = dict(width=5))
fig.update_xaxes(title_text = "Bfield",title_font = {"size": 25},title_standoff = 12, 
                    tickfont = dict(size=25))
fig.update_yaxes(title_text = "jcrit",title_font = {"size": 25},title_standoff = 12, 
                    tickfont = dict(size=25) ,type="log", exponentformat = 'e')
fig.show()


各向异性公式，0.1T ~ 1T, theta:0 ~ pi

In [ ]:
jt =  np.zeros((10, 100))

for i in range(10):
    # j0 = Jcrit_theta((i+1)/10, 0) 
    for j in range(100):
        Bx, By = (i+1)/10*np.sin(j*pi/99), (i+1)/10*np.cos(j*pi/99)
        jt0 = Jcrit_theta(Bx, By)
        jt = jt.at[i,j].set(jt0)

print(jt)
fig = go.Figure()
for i in range(10):
    fig.add_scatter(x = np.arange(0, pi, pi/100), y = jt[i], 
                    name = 'jcrit_theta_{}T'.format((i+1)/10), line = dict(width=5))
fig.update_xaxes(title_text = "theta",title_font = {"size": 25},title_standoff = 12, 
                    tickfont = dict(size=25))
fig.update_yaxes(title_text = "jcrit",title_font = {"size": 25},title_standoff = 12, 
                    tickfont = dict(size=25) , exponentformat = 'e')#,type="log")
fig.show()

各向异性公式不同J0测试（应变公式温度变化）, 以B=0.01T作为j0的参考


In [ ]:
js, jtx, jty = np.zeros((18, 100)), np.zeros((18, 100)), np.zeros((18, 100))
strain = 0
B0 = 0.02
jc0 = 3e10

for i in range(18):
    jc = Jcrit_strain((i+1)*4, 0.02, strain)
    for j in range(100):
        js = js.at[i,j].set(Jcrit_strain((i+1)*4, (j+1)/100, 0.004))
        Bx, By = (j+1)/100, 0
        jtx0 = Jcrit_theta(Bx, By, B0, jc)
        Bx, By = 0, (j+1)/100
        jty0 = Jcrit_theta(Bx, By, B0, jc)
        jtx = jtx.at[i, j].set(jtx0)
        jty = jty.at[i, j].set(jty0)

print(js)
print(jtx)
print(jty)
color = ['red', 'blue', 'green', 'yellow']
fig = go.Figure()
for i in range(4):
    fig.add_scatter(x = np.arange(0.01, 1.01, 1/100), y = js[i+4], 
                        name = 'jcrit_strain_{}K'.format((i+5)*4), line = dict(dash='longdash',width=5,color=color[i]))
    fig.add_scatter(x = np.arange(0.01, 1.01, 1/100), y = jtx[i+4], 
                        name = 'jcrit_theta_X_{}K'.format((i+5)*4), line = dict(width=5,color=color[i]))
    fig.add_scatter(x = np.arange(0.01, 1.01, 1/100), y = jty[i+4], 
                        name = 'jcrit_theta_y_{}K'.format((i+5)*4), line = dict(dash='dashdot',width=5,color=color[i]))
fig.update_xaxes(title_text = "Bfield",title_font = {"size": 25},title_standoff = 12, 
                    tickfont = dict(size=25))
fig.update_yaxes(title_text = "jcrit",title_font = {"size": 25},title_standoff = 12, 
                    tickfont = dict(size=25), exponentformat = 'e' )#,type="log")
fig.update_layout(legend=dict(x=0.8, y=1, font = dict(size=24)))
fig.show()


添加应变